In [ ]:
!pip install python-docx pypandoc

from google.colab import drive
import os
import pandas as pd
import re
from docx import Document
import pypandoc
import glob # To find files matching a pattern

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:


# Path to the folder containing the original Word files
folder_path = '/content/drive/MyDrive/worker_comp/DSI2'  # Path to folder contains original docs

In [ ]:
#convert doc files to docx

# Get all file names in the folder
file_names = os.listdir(folder_path)

# Filter out only the doc Word files (assuming Word files have extension .doc or .docx)
doc_word_files = [file_name for file_name in file_names if file_name.endswith('.doc')]

# Function to convert .doc to .docx
def convert_doc_to_docx(doc_path):
    docx_path = doc_path + 'x'  # Just append 'x' to the .doc file name to create .docx
    pypandoc.convert_file(doc_path, 'docx', outputfile=docx_path)
    return docx_path

for file_name in doc_word_files:
  doc_path = os.path.join(folder_path, file_name)
  docx_path = convert_doc_to_docx(doc_path)

In [ ]:
# Function to extract dates from the first page of a document
def extract_dates(doc_path):
    if doc_path.endswith('.doc'):
        doc_path = convert_doc_to_docx(doc_path)

    document = Document(doc_path)
    text = ''

    # Extract text from the first page
    for paragraph in document.paragraphs:
        text += paragraph.text + '\n'

    # Search for dates in the format "Month Day, Year" (e.g., "April 23, 2014")
    dates = re.findall(r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}', text)

    # Assuming the first date found is the beginning date, and the second is the end date
    beginning_date = dates[0] if dates else 'N/A'
    end_date = dates[1] if len(dates) > 1 else 'N/A'

    return beginning_date, end_date

In [ ]:
# Path to the folder containing the docx files
folder_path = '/content/drive/MyDrive/worker_comp/DSI2'

In [ ]:
import os
import re
import pandas as pd
from docx import Document
# Assuming convert_doc_to_docx exists if needed, or handle .doc separately.
# from your_module import convert_doc_to_docx # Example import

# --- Configuration ---
INPUT_DIRECTORY = '/content/drive/MyDrive/worker_comp/DSI2' # <<< CHANGE THIS to the folder containing your DOCX files
OUTPUT_EXCEL_FILE = '/content/drive/MyDrive/worker_comp/output_extracted_data.xlsx' # <<< CHANGE THIS (optional) to your desired output filename

# --- Section Definitions ---
# Define the sections and their variations (as provided previously)
SECTION_VARIATIONS = {
    "Appearances": ['A P E A R A N C E S', 'A P P E A R A N C E', 'A P P E A R A N C ES', 'A P P E A R A N C E S', 'A P P E A R A N C E S_2', 'A P P E A R A NC E S', 'A P P E A R AN C E S', 'A P P E A R ANCES', 'A P P E A RA N C E S', 'A P P E AR A N C E S', 'A P P E R A N C E S', 'A P P EA R A N C E S', 'A P P EA R AN C E S', 'A P P  E A R A N C E S', 'APPEARANCES', 'APPERARANCES'],
    "Stipulations": ['ADDITIONAL STIPULATIONS', 'COMMON STIPULATIONS', 'FACTUAL STIPULATIONS', 'GENERAL STIPULATIONS', 'JOINT STIPULATIONS', 'JURISDICTIONAL AND STANDARD STIPULATIONS', 'PROCEDURAL STIPULATION', 'S T I P U L A T I O N S', 'STIPUALTIONS', 'STIPULATED FACTS', 'STIPULATION', 'STIPULATION OF FACTS', 'STIPULATION OF THE FACTS', 'STIPULATION OF UNDISPUTED FACTS', 'STIPULATIONS', 'STIPULATIONS BETWEEN THE PARTIES', 'STIPULATIONS BY DEFENDANTS', 'STIPULATIONS COMMON TO BOTH CLAIMS', 'STIPULATIONS OFFERED BY DEFENDANT', 'STIPULATIONS OF FACT', 'STIPULATIONS OF FACTS', 'STIPULATIONS POST HEARING', 'STIPULATIONS_2', 'STIPULATONS', 'SUPPLEMENTAL STIPULATION'],
    "Exhibits": ['ADDITIONAL EXHIBITS', 'ADMITTED EXHIBITS', 'DEMONSTRATIVE EVIDENCE', 'DOCUMENTARY EVIDENCE', 'E X H I B I T S', 'EVIDENCE ADMITTED', 'EXBIBITS', 'EXHIB', 'EXHIBIT', 'EXHIBIT A', 'EXHIBITS', 'EXHIBITS ADMITTED', 'EXHIBITS AND DEPOSITIONS', 'EXHIBITS INTRODUCED AT THE HEARING', 'EXHIBITS INTRODUCED INTO THE RECORD', 'EXHIBITS MATTERS', 'EXHIBITS RECEIVED INTO EVIDENCE', 'EXHIBITS_2', 'EXHIBITS_3', 'HEARING DOCUMENTS', 'HEARING EXHIBITS', 'MEDICAL RECORDS ADMITTED INTO EVIDENCE', 'OTHER EXHIBITS', 'OTHER EXHIBITS INTRODUCED AT THE HEARING', 'STIPULATED DOCUMENTS', 'STIPULATED EVIDENCE', 'STIPULATED EXHIB', 'STIPULATED EXHIBIT', 'STIPULATED EXHIBIT I', 'STIPULATED EXHIBIT II', 'STIPULATED EXHIBITS', 'STIPULATED EXHIBITS_2', 'STIPULATED RECORDS', 'SUPPLEMENTAL EXHIBIT'],
    "Issues": ['ADDING NEW ISSUE FOR HEARING', 'ADDITIONAL ISSUES INCLUDE', 'CONTESTED ISSUES', 'CONTESTED ISSUES OF THE DEFENDANT', 'CONTESTED ISSUES OF THE PLAINTIFF', 'DEFENDANT GABRIEL BUILDERS ISSUES', 'DEFENDANT TRAVELERS ISSUES', 'DEFENDANTS ISSUES', 'ISSUE', 'ISSUE FOR DECISION', 'ISSUE FOR DETERMINATION', 'ISSUE FOR DETERMINATION INCLUDES', 'ISSUE FOR HEARING', 'ISSUE PRESENTED', 'ISSUE TO BE DECIDED', 'ISSUE TO BE DETERMINED', 'ISSUES', 'ISSUES DETERMINED', 'ISSUES FOR DECISION', 'ISSUES FOR DETERMINATION', 'ISSUES FOR DETERMINATION INCLUDE THE FOLLOWING', 'ISSUES FOR DETERMINATON', 'ISSUES FOR HEARING', 'ISSUES FOR RESOLUTION', 'ISSUES IN DISPUTE', 'ISSUES OF DEFENDANTS', 'ISSUES OF PLAINTIFF', 'ISSUES PRESENTED', 'ISSUES TO BE DECIDED', 'ISSUES TO BE DETERMINED', 'ISSUES TO BE DETRMINED', 'ISSUES TO BE DTERMINED', 'ISSUES TO BE RESOLVED', 'ISSUES TO BE  DETERMINED', 'ISSUES_2', 'ISSUED TO BE DETERMINED', 'ISSURES FOR DETERMINATION', 'ISSUES TO BE DETERMINED', 'ITEMS TO BE DETERMINED', 'MEDICAL ISSUE TO BE DETERMINED', 'PLAINTIFF ISSUES', 'PROPOSED ISSUE FOR DETERMINATION', 'PROPOSED ISSUES FOR DETERMINATION', 'PSYCHOLOGICAL ISSUE', 'QUESTIONS PRESENTED', 'RULING ON ADDITIONAL ISSUE RAISED BY PLAINTIFF IN CONTENTIONS', 'RULING ON ISSUES PRESENTED', 'RULINGS ON ISSUES PRESENTED', 'STATEMENT OF ISSUES', 'STIPULATED CONTESTED ISSUES', 'STIPULATED ISSUES', 'STIPULATED ISSUES PRESENTED', 'WAGE ISSUE'],
    "Findings of Fact": ['ADDITIONAL FINDING OF FACT', 'ADDITIONAL FINDINGS OF FACT', 'F I N D I N G S  OF FACT', 'FACTS', 'FINDING AND FACTS', 'FINDING OF FACT', 'FINDING OF FACTS', 'FINDINGS  OF FACT', 'FINDINGS FACT', 'FINDINGS OF FACT', 'FINDINGS OF FACT BASED ON THE ENTIRE RECORD', 'FINDINGS OF FACT FACTS', 'FINDINGS OF FACT_2', 'FINDINGS OF FACTS', 'FINDINGS OF THE FACT', 'FINDINGS OF THE FACTS', 'FINDINGS OFFACT', 'FINDINGSOFFACT', 'FINDINS OF FACT', 'FINDNGS OF FACT', 'FINDS OF FACTS', 'FURTHER FINDINGS OF FACT BY CONSENT', 'STATEMENT OF FACTS', 'STATEMENT OF STIPULATED FACTS', 'STATEMENT OF THE FACTS', 'STIPULATED FACTUAL FINDINGS', 'STIPULATED FINDINGS OF FACT', 'STIPULATED FINDINGS OF FACTS'],
    "Conclusions of Law": ['AGREED UPON CONCLUSIONS OF LAW', 'CONCLUSION AT LAW', 'CONCLUSION OF LAW', 'CONCLUSION OF THE LAW', 'CONCLUSIONS', 'CONCLUSIONS LAW', 'CONCLUSIONS OF LAW', 'CONCLUSIONS OF LAW PERTAINING TO THE ASBESTOS CLAIM', 'CONCLUSIONS OF LAW PERTAINING TO THE EYE CLAIM', 'CONCLUSIONS OF LAW_2', 'CONCLUSIONS OF L A W', 'CONCLUSIONS  OF  LAW', 'CONCLUSIONS  OF LAW', 'CONCLUSIONSOFLAW', 'CONCULSIONS OF LAW', 'CONLCUSIONS OF LAW', 'CONSLUSIONS OF LAW', 'CONCLUSUONS OF LAW', 'STIPULATED CONCLUSIONS OF LAW'],
    "Order/Award": ['AWARD & ORDER', 'AWARD AND ORDER', 'AWARD  &  ORDER','A W A R D', 'A W A R D S', 'A W A R D_2', 'A W A RD', 'A W AR D', 'A  W A R D', 'A WA R D', 'A W A  D', 'A W A R  D', 'AWARD', 'INTERLOCATORY AWARD', 'O R D E R', 'O R D E R E D', 'O R D ER', 'O R DE R', 'ORDER', 'ORDER BY DEPUTY COMMISSIONER LORI GAINES', 'ORDERED', 'ORDERS','O R D E R & A W A R D', 'O R D E R  &  A W A RD', 'O R D E R  &  A W A R D', 'ORDER & AWARD', 'ORDER AND AWARD'],
    "Combined Sections": [ 'CONCLUSIONS AND ORDER', 'CONCLUSIONS OF LAW AND AWARD', 'FINDING OF FACT AND CONCLUSION OF LAW', 'FINDINGS OF FACT & CONCLUSIONS OF LAW', 'FINDINGS OF FACT AND CONCLUSIONS OF LAW',  'STIPULATIONS & FINDINGS OF FACT', 'STIPULATIONS AND FINDINGS OF FACT', 'STIPULATIONS CONSENT ORDER AND AWARD'],
    "Evidentiary Rulings/Matters": ['ADDITIONAL EVIDENTIARY PLEADINGS TO BE INCLUDED IN THE RECORD', 'ADDITIONAL EVIDENTIARY RULING', 'ADDITIONAL EVIDENTIARY RULINGS', 'EVIDENCIARY MATTER', 'EVIDENCIARY RULING', 'EVIDENCIARY RULINGS', 'EVIDENRTIARY MATTERS', 'EVIDENTARY MATTERS', 'EVIDENTARY RULING', 'EVIDENTIARY AND OTHER RULINGS', 'EVIDENTIARY DOCUMENTS', 'EVIDENTIARY ISSUE', 'EVIDENTIARY ISSUES', 'EVIDENTIARY MATER', 'EVIDENTIARY MATTER', 'EVIDENTIARY MATTERS', 'EVIDENTIARY MATTTERS', 'EVIDENTIARY NOTE', 'EVIDENTIARY RULES', 'EVIDENTIARY RULING', 'EVIDENTIARY RULING REGARDING THE TESTIMONY OF AL GORROD', 'EVIDENTIARY RULINGS', 'EVIDENTIARY RULINGS ON OBJECTIONS RAISED DURING DEPOSITION', 'EVIDENTIARY RULINGS ON OBJECTIONS RAISED DURING DEPOSITIONS', 'EVIDENTIARY RULINGS_2', 'EVIDENTIATY MATTERS', 'EVIDENTIRAY MATTERS', 'OBJECTIONS', 'OBJECTIONS AND ADMISSION OF EXHIBITS', 'OBJECTIONS AND EVIDENTIARY RULINGS', 'OBJECTIONS RAISED DURING DEPOSITIONS', 'RULING ON EVIDENCE MATTERS', 'RULING ON EVIDENTIARY ISSUE', 'RULING ON EVIDENTIARY MATTER', 'RULING ON EVIDENTIARY MATTERS', 'RULING ON EVIDENTIARY OBJECTIONS', 'RULING ON HEARSAY OBJECTIONS', 'RULING ON OBJECTIONS', 'RULINGS ON DEPOSITION OBJECTIONS', 'RULINGS ON EVIDENTIARY MATTERS', 'RULINGS ON EVIDENTIARY MATTERS AND FURTHER ORDERS', 'RULINGS ON OBJECTIONS AND ADMISSION OF EXHIBITS', 'RULINGS ON THE EVIDENCE', 'RULES ON EVIDENTIARY MATTERS'],
    "Depositions/Testimony": ['DEPOSITION', 'DEPOSITION OF CAROL HARRIS', 'DEPOSITION OF KELLY LANCE', 'DEPOSITION OF STEPHEN CARPENTER', 'DEPOSITION TAKEN', 'DEPOSITION TESTIMONY', 'DEPOSITION TRANSCRIPTS', 'DEPOSITIONS', 'DEPOSITIONS UPON ORAL EXAMINATION', 'DEPOSITIONS_2', 'DEPOSITONS', 'DEPONENTS', 'DESPOSITIONS', 'EXPERT OPINION', 'EXPERT TESTIMONY', 'EXPERT TESTIMONY RULING', 'EXPERT WITNESS WRITTEN RESPONSES', 'EXPERTS', 'LIST OF PERSONS DEPOSED', 'MEDICAL OPINION CORRESPONDENCE SUBMITTED IN LIEU OF DEPOSITIONS', 'MEDICAL QUESTIONNAIRE RESPONSES', 'MEDICAL QUESTIONNAIRES', 'MEDICAL WITNESSES', 'PERSONS DEPOSED', 'RESPONSES TO MEDICAL QUESTIONNAIRE & DEPOSITION', 'TESTIMONIAL EVIDENCE', 'TESTIMONY BY DEPOSITION', 'WITNESSES'],
    "Procedural History/Matters": ['ADDITIONAL PROCEDURAL HISTORY', 'BACKGROUND', 'CASE HISTORIES', 'CASE HISTORY', 'FACTUAL AND PROCEDURAL HISTORY', 'HISTORY OF CASE', 'INTRODUCTION', 'INTRODUCTORY PARAGRAPH', 'JUDICIAL HISTORY', 'PARTIAL PROCEDURAL HISTORY', 'PRELIMINARY MATTERS', 'PRELIMINARY RULINGS', 'PRIOR OPINION AND AWARDS', 'PROCEDURAL AND EVIDENTIARY ISSUES', 'PROCEDURAL BACKGROUND', 'PROCEDURAL HISTORY', 'PROCEDURAL HISTORY AND POSTURE', 'PROCEDURAL HISTORY OF CLAIM', 'PROCEDURAL HISOTRY', 'PROCEDURAL MATTERS', 'PROCEDURAL MATTERS AND EVIDENTIARY RULING', 'PROCEDURAL POSTURE', 'PROCEDURAL RULING', 'PROCEDURAL RULINGS', 'RELEVANT PROCEDURAL HISTORY', 'STATEMENT OF THE CASE'],
    "Motions/Rulings on Motions": ['EVIDENTIARY AND RULINGS ON MOTIONS', 'EVIDENTIARY MOTION', 'MOTION', 'MOTION FOR LEAVE TO SUBMIT ADDITIONAL EVIDENCE', 'MOTION IN LIMINE', 'MOTION RULING', 'MOTION TO ADMIT EVIDENCE', 'MOTION TO COMPEL APPROVAL OF MEDICAL TREATMENT', 'MOTION TO DISMISS', 'MOTION TO DISMISS RULING', 'MOTION TO ENLARGE TIME', 'MOTION TO EXCLUDE EXPERT TESTIMONY', 'MOTION TO ORDER SURGERY', 'MOTION TO RECONSIDER', 'MOTION TO STRIKE', 'MOTION TO SUPPLEMENT RECORD', 'MOTION TO TAKE JUDICIAL NOTICE', 'MOTIONS', 'MOTIONS AND EVIDENTIARY RULINGS', 'OUTSTANDING MOTIONS', 'PENDING MOTIONS', 'PRELIMINARY MOTIONS', 'PRETRIAL MOTIONS', 'PROCEDURAL MATTERS AND MOTIONS RULINGS', 'RULING ON GUARDIAN AD LITEM MOTION', 'RULING ON MOTION', 'RULING ON MOTION TO DISMISS', 'RULING ON MOTION TO EXCLUDE DISPUTED MEDICAL RECORDS', 'RULING ON MOTION TO STRIKE', 'RULINGS ON MOTIONS', 'RULINGS ON MOTIONS AND EVIDENTIARY MATTERS', 'SUMMARY JUDGMENT RULING', 'SUPPLEMENTAL RULING ON MOTION'],
    "Commissioner Details": ['CHIEF DEPUTY COMMISSIONER', 'COMMISSIONER', 'DEPUTY COMMISSIONER', 'DEPUTY COMMISSIONER MARY CLAIRE BROWN', 'DEPUTY COMMISSIONER RONNIE ROWELL', 'DEPUTY COMMISSIONER SAM SCUDDER', 'DEPUTY COMMISSIONER_2', 'DEPUTY COMMISIONER', 'DEPUTY COMMISSISONER', 'DEPUTY OMMISSIONER', 'DEPUTYCOMMISSIONER', 'INTERIM CHIEF DEPUTY COMMISSIONER', 'OPINION AND AWARD BY CHIEF DEPUTY COMMISSIONER MELANIE WADE GOODWIN', 'OPINION AND AWARD BY DEPUTY COMMISSIONER', 'OPINION AND AWARD BY DEPUTY COMMISSIONER SAM SCUDDER', 'OPINION & AWARD BY DEPUTY COMMISSIONER TIFFANY SMITH', 'SENIOR DEPUTY COMMISSIONER'],
    "NC Industrial Commission Reference": ['BEFORE THE NORTH CAROINA INDUSTRIAL COMMISSION', 'BEFORE THE NORTH CAROLINA INDUSTRIAL COMMISSION', 'INDUSTRIAL COMMISSION', 'NORTH CAROLINA INDUSTRIAL COMMISSION', 'NORTH CAROLINA INDUSTRIAL COMMISION', 'NORTHCAROLINAINDUSTRIALCOMMISSION', 'THE NORTH CAROLINA INDUSTRIAL COMMISSION'],
    "Consent Order/Opinion/Award": ['AGREEMENT', 'AGREEMENT OF THE PARTIES', 'CONSENT', 'CONSENT FINDINGS OF FACT', 'CONSENT OPINION AND AWARD', 'CONSENT OPINION AND AWARD FOR PAYMENT OF DEATH BENEFITS', 'CONSENT OPINION  AWARD', 'CONSENT ORDER', 'CONSENT ORDER & AWARD', 'CONSENT ORDER AND AWARD', 'CONSENT PROPOSED OPINION AND AWARD', 'FINDINGS BY CONSENT OF THE PARTIES', 'FURTHER FINDINGS OF FACT BY CONSENT', 'ORDER AND AWARD BY CONSENT'],
    "Opinion and Award Title": ['AMENDED OPINION AND AWARD', 'AMENDED OPINION & AWARD BY DEPUTY COMMISSIONER TIFFANY SMITH', 'OPINION AND AWARD', 'OPINION AND AWARD BY', 'OPINION AND AWARD FOR PAYMENT OF RATING', 'OPINION AND AWARD ON EXPEDITED MEDICAL MOTION PURSUANT TO', 'OPINION & AWARD BY DEPUTY COMMISSIONER TIFFANY SMITH', 'PROPOSED OPINION AND AWARD', 'TO THE INTERLOCUTORY OPINION AND AWARD AND AMENDED INTERLOCUTORY OPINION AND AWARD'],
    "Medical Related": ['ADDITION OF VOCATIONAL REHABILITATION ISSUE BY DEFENDANT', 'ATTENDANT CARE', 'AVERAGE WEEKLY WAGE', 'CAUSAL RELATIONSHIP', 'DATE OF INJURY', 'DENTAL CONDITION', 'DISABLITY', 'INJURY BY ACCIDENT', 'MEDICAL EVIDENCE', 'MEDICAL HISTORY', 'MEDICAL RECORDS', 'MEDICAL TREATMENT', 'PERIODIC SPINE SPECIALIST EVALUATION', 'PHYSICAL INJURIES', 'PRIOR MEDICAL TREATMENT', 'PSYCHOLOGICAL', 'REIMBURSEMENT', 'SALTWATER TANK THERAPY', 'THE BED', 'THE SHOWER', 'TWICE WEEKLY MASSAGE THERAPY', 'UROLOGY REFERRAL'],
    "Parties/Counsel/Attorneys": ['APPOINTMENT OF GUARDIAN AD LITEM', 'ATTORNEY FOR DEFENDANTS', 'ATTORNEY FOR EMPLOYEE', 'ATTORNEY FOR PLAINTIFF', 'COUNSEL FOR DEFENDANTS', 'COUNSEL FOR PLAINTIFF', 'DEFENDANT', 'DEFENDANT AEROTEK', 'DEFENDANT BECTON DICKINSON', 'DEFENDANT PICKETT', 'DEFENDANT PRA', 'DEFENDANT TRAVELERS', 'DEFENDANTS', 'P A R T I E S', 'PLAINTIFF', 'ROBERTS LAW FIRM', 'SCUDDER & HEDRICK', 'WARD BLACK LAW', 'YOUNG MOORE AND HENDERSON'],
    "Other/Miscellaneous": ['ADDITIONAL EVIDENCE', 'ADDITIONAL EVIDENCE SUBMITTED', 'ADMINISTRATIVE ORDER', 'AMENDED', 'APPROVED', 'APPROVED WITH CONSENT OF THE PARTIES', 'CERTIFICATE OF SERVICE', 'CELESTE HARRIS', 'COMMENT', 'COMMENTARY', 'CONTENTIONS', 'CONTENTIONS OF THE PARTIES', 'COUNTY OF MECKLENBURG', 'COURT', 'CREDIBILITY', 'DOBBS BUILDING', 'DOCTOR QUESTIONNAIRE', 'EMPLOYMENT HISTORY', 'EVIDENCE', 'FURTHER RULING', 'GENERAL', 'HOLDINGS BY THE FULL COMMISSION WHICH HAVE NOT BEEN APPEALLED', 'IMOCO', 'INSURER CNA INSURANCE', 'IT APPEARING TO THE UNDERSIGNED THAT', 'IT IS HEREBY ORDERED', 'IT IS FURTHER ORDERED', 'JUDICIAL NOTICE', 'JURISDICTIONAL DISCUSSION', 'NORTH CAROLINA INSURANCE GUARANTY ASSOCIATION', 'NOTARY PUBLIC', 'NOW WITHDRAWN', 'OFFER OF PROOF', 'OFFERS OF PROOF', 'OTHER HOME MODIFICATIONS', 'OTHER ISSUES', 'OTHER MATTERS', 'OTHER RULINGS', 'OTHER STATE INSURANCE', 'PART THREE OTHER STATE INSURANCE', 'PREAMBLE', 'PREVIOUS AWARDS AND CONSENT ORDERS', 'RELEVANT TO THE INSTANT ISSUES', 'RULING', 'RULING ON JUDICIAL NOTICE', 'RULING ON SCOPE OF HEARING', 'SANCTIONS', 'SETTLEMENT TERMS', 'STATE OF NORTH CAROLINA', 'STATEMENTS', 'SUBSEQUENT TO THE CLOSING OF THE EVIDENTIARY RECORD', 'TAMMY NANCE', 'THE ALLEGED ACCIDENT AND NOTICE TO EMPLOYER', 'THOMAS VAN CAMP', 'WARD NORTH AMERICA IS DISMISSED AS A PARTY DEFENDANT AND TIG PREMIER INSURANCE COMPANY IS ADDED AS A PARTY DEFENDANT', 'WARD NORTH AMERICA PAYS THE PRO SE PLAINTIFF BENEFITS OUTSIDE THE JURISDICTION OF THE INDUSTRIAL COMMISSION', 'WITNESSETH']
}

# Create a reverse map: header variation -> standardized category name
# Normalize by stripping whitespace and converting to uppercase for robust matching
header_to_category_map = {}
for category, variations in SECTION_VARIATIONS.items():
    for variation in variations:
        normalized_variation = variation.strip().upper()
        if normalized_variation: # Avoid empty strings
            header_to_category_map[normalized_variation] = category

# --- Helper Functions ---

def extract_dates(doc_path):
    """
    Extracts beginning and end dates (Month Day, Year format) from the document.
    Placeholder: Adapt this function based on where dates reliably appear in your docs.
    This version scans the whole document text.
    """
    try:
        document = Document(doc_path)
        text = '\n'.join([para.text for para in document.paragraphs])

        # Search for dates in the format "Month Day, Year" (e.g., "April 23, 2014")
        # This regex is basic, adjust if date formats vary significantly
        dates_found = re.findall(
            r'\b(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2},\s+\d{4}\b',
            text
        )

        # Prioritize dates found earlier in the document if multiple exist
        beginning_date = dates_found[0] if dates_found else 'N/A'
        # Often the primary date is the first one. You might need more complex logic
        # if start/end dates are explicitly labeled or always the first two.
        # For simplicity, this example just takes the first found date.
        # end_date = dates_found[1] if len(dates_found) > 1 else 'N/A'
        # date_str = f"{beginning_date} - {end_date}" if end_date != 'N/A' else beginning_date

        return beginning_date # Return only the first date found as the primary 'Date'

    except Exception as e:
        print(f"Warning: Could not extract dates from {os.path.basename(doc_path)}: {e}")
        return "N/A"

def process_word_doc_specific_sections(file_path):
    """
    Reads a .docx file, identifies specific sections based on predefined header lists,
    groups the text under standardized category names, and returns the data as a dictionary.
    Sections end at the next recognized header or a line with '***...'.
    """
    try:
        # --- 1. Extract Case ID from filename ---
        base_name = os.path.basename(file_path)
        case_id = os.path.splitext(base_name)[0] # Removes extension
        case_id = re.sub(r'\.(docx|doc)$', '', case_id, flags=re.IGNORECASE) # Clean potential original extension
        # print(f"Processing Case ID: {case_id}") # Moved to main loop

        # --- 2. Read Document ---
        try:
            # Add .doc conversion logic here if needed before opening
            document = Document(file_path)
        except Exception as e:
            if "File is not a zip file" in str(e) or "Package not found" in str(e):
                 print(f"Error opening {base_name}: Invalid .docx format or corrupted. Original error: {e}")
            else:
                 print(f"Error opening document {base_name}: {e}")
            return None # Skip this file

        # --- 3. Extract Date ---
        date_str = extract_dates(file_path)

        # --- 4. Extract Specific Sections ---
        asterisk_delimiter_regex = re.compile(r"^\s*\*{2,}\s*$")
        grouped_sections = {} # Stores lists of text blocks for each category
        current_category_key = "Preamble" # Default category
        current_text_lines = []

        for para in document.paragraphs:
            para_text_stripped = para.text.strip()
            para_text_upper = para_text_stripped.upper() # Use for matching headers

            if not para_text_stripped: # Skip empty paragraphs
                continue

            recognized_category = header_to_category_map.get(para_text_upper)
            asterisk_match = asterisk_delimiter_regex.match(para_text_stripped)

            # Check if it's a recognized header
            if recognized_category:
                # Store previous section's text
                if current_category_key and current_text_lines:
                    section_text = "\n".join(current_text_lines).strip()
                    if section_text:
                        grouped_sections.setdefault(current_category_key, []).append(section_text)

                # Start new section
                current_category_key = recognized_category
                current_text_lines = []

            # Check if it's an asterisk delimiter
            elif asterisk_match:
                # Store previous section's text
                if current_category_key and current_text_lines:
                    section_text = "\n".join(current_text_lines).strip()
                    if section_text:
                        grouped_sections.setdefault(current_category_key, []).append(section_text)

                # Stop collecting until next header
                current_category_key = None
                current_text_lines = []

            # Otherwise, it's regular text content
            elif current_category_key: # Only collect if we are 'inside' a section
                current_text_lines.append(para.text) # Append original text

        # Store the last section's text
        if current_category_key and current_text_lines:
            section_text = "\n".join(current_text_lines).strip()
            if section_text:
                 grouped_sections.setdefault(current_category_key, []).append(section_text)

        # --- 5. Prepare Final Data Dictionary for this file ---
        final_data = {
            "Case ID": case_id,
            "Date": date_str,
        }

        # Join the text blocks for each category found in this document
        for category, text_blocks in grouped_sections.items():
            if text_blocks: # Ensure there's content
                 valid_blocks = [block for block in text_blocks if block] # Filter empty blocks just in case
                 if valid_blocks:
                    # Join multiple occurrences of the same category type with double newlines
                    final_data[category] = "\n\n".join(valid_blocks)

        # Clean up potential empty preamble
        if "Preamble" in final_data and not final_data["Preamble"]:
            del final_data["Preamble"]
        # Add other default fields if necessary, e.g., filename
        # final_data["Source File"] = base_name

        return final_data

    except FileNotFoundError:
        # This case should ideally be handled by the main loop checking os.path.exists
        print(f"Error: Input file not found at {file_path}")
        return None
    except ImportError:
         # Should be caught at the start if pandas/docx aren't installed
         print("Error: Required library (pandas, python-docx) not found.")
         raise # Re-raise to stop execution
    except Exception as e:
        print(f"An unexpected error occurred processing {os.path.basename(file_path)}: {e}")
        # import traceback
        # print(traceback.format_exc()) # Uncomment for detailed debugging
        return None # Skip this file on error

# --- Main Execution ---
if __name__ == "__main__":
    all_extracted_data = [] # List to hold dictionaries for each file

    # Check if input directory exists
    if not os.path.isdir(INPUT_DIRECTORY):
        print(f"Error: Input directory not found: {INPUT_DIRECTORY}")
        print("Please update the INPUT_DIRECTORY variable in the script.")

    else:
        print(f"Starting processing in directory: {INPUT_DIRECTORY}")
        # List all files in the directory
        try:
            files_to_process = [f for f in os.listdir(INPUT_DIRECTORY)
                                if os.path.isfile(os.path.join(INPUT_DIRECTORY, f))
                                and f.lower().endswith('.docx')] # Process only .docx files
        except Exception as e:
            print(f"Error listing files in directory {INPUT_DIRECTORY}: {e}")
            files_to_process = []

        if not files_to_process:
            print("No .docx files found in the specified directory.")
        else:
            print(f"Found {len(files_to_process)} .docx files to process.")

            # Process each file
            for filename in files_to_process:
                file_path = os.path.join(INPUT_DIRECTORY, filename)
                print(f"Processing: {filename}...")
                extracted_data = process_word_doc_specific_sections(file_path)

                if extracted_data:
                    all_extracted_data.append(extracted_data)
                    print(f"Finished: {filename}")
                else:
                    print(f"Skipped or failed: {filename}")

            # --- Create and Export Excel File ---
            if all_extracted_data:
                print(f"\nProcessed {len(all_extracted_data)} files successfully.")
                print("Creating DataFrame...")

                # Create DataFrame
                df = pd.DataFrame(all_extracted_data)

                # Define desired column order - Start with fixed columns, then add all possible section categories
                column_order = ["Case ID", "Date"]
                # Add all keys from SECTION_VARIATIONS to ensure all potential columns are included
                all_section_keys = list(SECTION_VARIATIONS.keys())
                # Add section keys only if they actually appeared in *any* document,
                # or uncomment the line above to force all potential columns.
                present_section_keys = [col for col in all_section_keys if col in df.columns]
                column_order.extend(sorted(present_section_keys)) # Sort section names alphabetically

                # Add any other columns that might have been generated but aren't sections (e.g., 'Preamble')
                other_cols = sorted([col for col in df.columns if col not in column_order])
                column_order.extend(other_cols)


                # Reindex DataFrame to ensure consistent column order and presence
                df = df.reindex(columns=column_order)

                print(f"Exporting data to Excel file: {OUTPUT_EXCEL_FILE}...")
                try:
                    # Export to Excel
                    df.to_excel(OUTPUT_EXCEL_FILE, index=False, engine='openpyxl')
                    print(f"Successfully created Excel file: {OUTPUT_EXCEL_FILE}")
                except Exception as e:
                    print(f"\nError writing to Excel file: {e}")
                    print("Ensure the file is not open in another application.")
                    print("You might need to install the 'openpyxl' library: pip install openpyxl")

            else:
                print("\nNo data was successfully extracted from any files.")

    print("\nScript finished.")


In [ ]:
import pandas as pd
import os # Import os module for better path handling

# --- Configuration ---
# Define the path to the Excel file
# <<< CHANGE THIS to the full path of your Excel file >>>
excel_file_path = '/content/drive/MyDrive/worker_comp/output_extracted_data.xlsx'

# Optional: Specify sheet name if not the first sheet
# sheet_name_to_read = 'Sheet1' # Or None to read the first sheet

try:
    # Check if the file exists before attempting to read
    if not os.path.exists(excel_file_path):
        raise FileNotFoundError(f"The file was not found at {excel_file_path}")

    # Read the Excel file into a pandas DataFrame
    # If you need a specific sheet, add the sheet_name argument:
    # df = pd.read_excel(excel_file_path, sheet_name=sheet_name_to_read)
    df = pd.read_excel(excel_file_path) # Reads the first sheet by default

    # Get the total number of rows (observations) in the DataFrame
    total_rows = len(df)

    print(f"Analyzing Excel file: {os.path.basename(excel_file_path)}")
    print(f"Total number of rows in the sheet: {total_rows}\n")
    print("Column Analysis (Non-Empty Observations):")
    print("-" * 60)
    print(f"{'Column Name':<30} | {'Non-Empty Count':<15} | {'% Non-Empty':<10}")
    print("-" * 60)

    if total_rows > 0:
        # Iterate through each column in the DataFrame
        for column_name in df.columns:
            # Get the count of non-null (non-empty) values in the column
            # .count() in pandas excludes NaN/None values
            non_empty_count = df[column_name].count()

            # Calculate the percentage of non-empty values
            percentage_non_empty = (non_empty_count / total_rows) * 100

            # Print the results for the current column, formatted
            print(f"{column_name:<30} | {non_empty_count:<15} | {percentage_non_empty:>9.2f}%")
    else:
        print("The Excel sheet is empty or contains only headers, no data rows to analyze.")

    print("-" * 60)

# --- Error Handling ---
except FileNotFoundError as fnf_error:
    print(f"Error: {fnf_error}")
    print("Please ensure the file exists and the path is correct.")
except pd.errors.EmptyDataError:
    # This error might not be directly raised by read_excel for an empty sheet,
    # but checking total_rows handles the case of an empty DataFrame.
    print(f"Error: The file at {excel_file_path} seems to contain no data or only headers.")
except ImportError:
     print("Error: The 'openpyxl' library is required to read Excel files.")
     print("Please install it using: pip install openpyxl")
except Exception as e:
    print(f"An error occurred while reading or processing the Excel file: {e}")
    # Consider adding more specific error handling if needed (e.g., for password-protected files)



In [ ]:
import pandas as pd
import os # Import os module for better path handling

# --- Configuration ---
# Input file: The Excel file containing the data (e.g., the output from the previous script)
# <<< CHANGE THIS to the full path of your INPUT Excel file >>>
input_excel_path =  '/content/drive/MyDrive/worker_comp/output_extracted_data.xlsx'

# Output file: The new Excel file with only selected columns
# <<< CHANGE THIS (optional) to your desired OUTPUT Excel filename >>>
output_filtered_excel_path = '/content/drive/MyDrive/worker_comp/filtered_extracted_data.xlsx'

# Optional: Specify sheet name if not the first sheet in the input file
# input_sheet_name = 'Sheet1' # Or None to read the first sheet

# Columns to keep in the new file (as specified by the user)
columns_to_keep = [
    'Case ID',
    'Date',
    'Preamble',
    'Appearances',
    'Stipulations',
    'Exhibits',
    'Issues',
    'Findings of Fact',
    'Conclusions of Law',
    'Order/Award'
    # Add any other columns from the input file you wish to retain
    # e.g., 'Source File' if it exists and you want it
]

# --- Processing ---
try:
    # Check if the input file exists
    if not os.path.exists(input_excel_path):
        raise FileNotFoundError(f"Input file not found at {input_excel_path}")

    # Read the input Excel file
    # Use sheet_name=input_sheet_name if reading a specific sheet
    # df_input = pd.read_excel(input_excel_path, sheet_name=input_sheet_name)
    df_input = pd.read_excel(input_excel_path) # Reads the first sheet by default
    print(f"Read {len(df_input)} rows from {os.path.basename(input_excel_path)}")
    print(f"Original columns: {df_input.columns.tolist()}")

    # Check which of the desired columns actually exist in the input file
    existing_columns_to_keep = [col for col in columns_to_keep if col in df_input.columns]
    missing_columns = [col for col in columns_to_keep if col not in df_input.columns]

    if missing_columns:
        print(f"\nWarning: The following requested columns were not found in the input file and will be skipped: {missing_columns}")

    if not existing_columns_to_keep:
          print("\nError: None of the requested columns were found in the input file. Cannot create the filtered file.")
    else:
        # Select only the existing columns that the user wants to keep
        df_filtered = df_input[existing_columns_to_keep]

        # Write the filtered DataFrame to the new Excel file
        # index=False prevents pandas from writing the DataFrame index as a column
        df_filtered.to_excel(output_filtered_excel_path, index=False, engine='openpyxl')
        print(f"\nSuccessfully created filtered Excel file: {output_filtered_excel_path}")
        print(f"Filtered file contains columns: {existing_columns_to_keep}")

# --- Error Handling ---
except FileNotFoundError as fnf_error:
    print(f"Error: {fnf_error}")
    print("Please ensure the input Excel file exists and the path is correct.")
except pd.errors.EmptyDataError:
     # This might not be directly raised by read_excel for an empty sheet,
     # but good to keep for robustness if the file structure is unusual.
    print(f"Error: The input file at {input_excel_path} appears to be empty or contain no data.")
except KeyError as e:
    print(f"Error: An unexpected issue occurred when selecting columns: {e}")
    print("This might happen if the column names in 'columns_to_keep' don't exactly match the input Excel file.")
except ImportError:
     print("Error: The 'openpyxl' library is required to read/write Excel '.xlsx' files.")
     print("Please install it using: pip install openpyxl")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

